In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, monotonically_increasing_id, when
from pyspark.sql.window import Window

# Tworzymy sesję Spark
spark = SparkSession.builder.appName("FASTQ_Reader_with_Record_ID").getOrCreate()

# Ścieżka do katalogu z FASTQ
fastq_dir = "/Volumes/databrics_2/default/fastq"

# Lista plików FASTQ
files = ["SRR16356246_1.fastq", "SRR16356246_2.fastq"]

for file_name in files:
    file_path = f"{fastq_dir}/{file_name}"
    
    # 1. Wczytanie pliku do DataFrame Spark
    lines_df = spark.read.text(file_path)
    
    # 2. Dodanie ciągłego numeru linii (line_number) - Ćwiczenie 8.2
    window = Window.orderBy(monotonically_increasing_id())
    lines_with_id = lines_df.withColumn("line_number", row_number().over(window) - 1)
    
    # 3. Określenie typu linii FASTQ - Ćwiczenie 8.3
    lines_typed = lines_with_id.withColumn(
        "line_type",
        when(col("line_number") % 4 == 0, "header")
        .when(col("line_number") % 4 == 1, "sequence")
        .when(col("line_number") % 4 == 2, "separator")
        .when(col("line_number") % 4 == 3, "quality")
    )
    
    # 4. Utworzenie identyfikatora odczytu / rekordu (record_id) - Ćwiczenie 8.4
    lines_with_record = lines_typed.withColumn(
        "record_id",
        (col("line_number") / 4).cast("integer")
    )
    
    # 5. Wyświetlenie pierwszych 16 linii z wszystkimi dodatkowymi kolumnami
    print(f"\nPlik: {file_name} - pierwsze 16 linii z line_number, line_type i record_id:")
    lines_with_record.show(16, truncate=False)
    
    # 6. Liczba linii i odczytów
    total_lines = lines_with_record.count()
    total_reads = total_lines // 4
    print(f"Liczba linii: {total_lines}")
    print(f"Liczba odczytów: {total_reads}\n")



Plik: SRR16356246_1.fastq - pierwsze 16 linii z line_number, line_type i record_id:


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+-----+-----------+---------+---------+
|value|line_number|line_type|record_id|
+-----+-----------+---------+---------+
+-----+-----------+---------+---------+



/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


Liczba linii: 0
Liczba odczytów: 0


Plik: SRR16356246_2.fastq - pierwsze 16 linii z line_number, line_type i record_id:
+-----+-----------+---------+---------+
|value|line_number|line_type|record_id|
+-----+-----------+---------+---------+
+-----+-----------+---------+---------+

Liczba linii: 0
Liczba odczytów: 0

